# Lab 1: Run AI Red Teaming Agent Scan on Your Agent

Welcome! This lab will guide you through running your first **AI Red Teaming Agent scan** to proactively test your Azure AI agent for safety vulnerabilities.

---




## What You'll Learn

By the end of this lab, you will:
- Understand how the AI Red Teaming Agent works
- Configure and authenticate with Azure AI Foundry
- Retrieve an existing agent from your project
- Set up an agent callback for testing
- Run an automated red team scan
- Analyze scan results and understand attack success rates

---


## What is AI Red Teaming?

The **AI Red Teaming Agent** is a powerful tool that helps organizations proactively find safety risks in generative AI systems. It simulates adversarial attacks by:

1. **Generating adversarial prompts** - Creates attack objectives across risk categories
2. **Sending prompts to the target** - Communicates with your AI agent or model
3. **Analyzing responses** - Evaluates if attacks successfully elicited unsafe content
4. **Generating security reports** - Provides metrics like Attack Success Rate (ASR)

> **Learn More**: [AI Red Teaming Agent Documentation](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/ai-red-teaming-agent)

![How AI Red Teaming Works](https://learn.microsoft.com/en-us/azure/ai-foundry/media/evaluations/red-teaming-agent/how-ai-red-teaming-works.png)

---

## Step 1: Suppress Warnings

First, let's suppress non-critical warnings to keep our output clean. The `confusables` module may generate syntax warnings that we can safely ignore.

In [ ]:
# Suppress SyntaxWarning from confusables module
import warnings
warnings.filterwarnings('ignore', category=SyntaxWarning, module='confusables')

## Step 2: Import Required Libraries

We'll import all the necessary libraries for this lab:

- **Azure AI Projects SDK** - For interacting with Azure AI Foundry agents
- **Azure AI Evaluation SDK** - For running red team scans
- **Azure Identity** - For authentication
- **Standard Python libraries** - For file handling and async operations

> **Note**: The AI Red Teaming Agent leverages [PyRIT (Python Risk Identification Tool)](https://github.com/Azure/PyRIT), Microsoft's open-source framework for AI red teaming.

In [ ]:
from typing import Optional, Dict, Any
import os
import time
from pathlib import Path
from dotenv import load_dotenv

# Azure imports
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy
from azure.ai.projects import AIProjectClient
import azure.ai.agents
from azure.ai.agents.models import ListSortOrder

print("✅ All libraries imported successfully!")

## Step 3: Load Environment Variables

Load the environment variables from the `.env` file. These should have been configured during the setup phase and include:

- **AZURE_AI_PROJECT_ENDPOINT** - Your Azure AI Foundry project endpoint URL
- **AZURE_AI_DEPLOYMENT_NAME** - The name of your deployed chat model
- **AZURE_EXISTING_AGENT_ID** or **AZURE_AI_AGENT_NAME** - The identifier for your agent

The code will search for the `.env` file in the current directory and parent directories.

In [ ]:
# Load environment variables from .env file
current_dir = Path(os.getcwd())
env_path = current_dir / ".env"

# Try loading from current directory first, then parent directories
if not env_path.exists():
    env_path = current_dir.parent / ".env"
if not env_path.exists():
    env_path = current_dir.parent.parent / ".env"

load_dotenv(dotenv_path=env_path)

print(f"✅ Environment variables loaded from: {env_path}")

## Step 4: Initialize Azure Credentials and Read Configuration

Set up Azure authentication using `DefaultAzureCredential` which supports multiple authentication methods:
- Azure CLI (`az login`)
- Managed Identity (when running in Azure)
- Environment variables
- Interactive browser (as fallback)

Then retrieve and validate the required configuration values.

In [ ]:
# Initialize Azure credentials
credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)

# Get AI project parameters from environment variables
project_endpoint = os.environ.get("AZURE_AI_PROJECT_ENDPOINT")
deployment_name = os.getenv("AZURE_AI_DEPLOYMENT_NAME")  
agent_id = os.environ.get("AZURE_EXISTING_AGENT_ID")
agent_name = os.environ.get("AZURE_AI_AGENT_NAME")

# Validate required environment variables
if not project_endpoint:
    raise ValueError("❌ AZURE_AI_PROJECT_ENDPOINT environment variable is not set!")
    
if not agent_id and not agent_name:
    raise ValueError("❌ Please set either AZURE_EXISTING_AGENT_ID or AZURE_AI_AGENT_NAME environment variable.")

print("✅ Azure credentials initialized successfully!")
print(f"📍 Project Endpoint: {project_endpoint}")
print(f"🤖 Agent ID: {agent_id if agent_id else 'Will lookup by name: ' + agent_name}")

## Step 5: Create AI Project Client and Retrieve Agent

Connect to your Azure AI Foundry project and retrieve the agent you want to test. If only the agent name is provided, the code will search through all agents in the project to find the matching one.

In [ ]:
# Create AI Project Client
project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)

# Look up the agent by name if agent ID is not provided
if not agent_id and agent_name:
    print(f"🔍 Searching for agent with name: {agent_name}")
    for agent_item in project_client.agents.list_agents():
        if agent_item.name == agent_name:
            agent_id = agent_item.id
            print(f"✅ Found agent: {agent_name} (ID: {agent_id})")
            break
            
if not agent_id:
    raise ValueError("❌ Agent ID not found. Please provide a valid agent ID or name.")
    
# Retrieve the agent
agent = project_client.agents.get_agent(agent_id)

# Use model from agent if deployment name not explicitly provided
if not deployment_name:
    deployment_name = agent.model
    
print(f"\n✅ Agent retrieved successfully!")
print(f"  - Agent ID: {agent.id}")
print(f"  - Agent Name: {agent.name}")
print(f"  - Using Model: {deployment_name}")

## Step 6: Create a Thread for Agent Communication

Create a conversation thread that will be used to communicate with the agent during red team testing. A thread maintains the conversation context and history.

In [ ]:
# Create a thread for agent communication
thread = project_client.agents.threads.create()

print(f"✅ Conversation thread created: {thread.id}")

## Step 7: Define Agent Callback Function

Create a callback function that the Red Team Agent will use to interact with your agent. This function:

1. Sends the adversarial query to the agent
2. Creates a run to process the query
3. Polls for completion
4. Returns the agent's response

This callback simulates how a user would interact with your agent, allowing the Red Team to test for vulnerabilities.

In [ ]:
def agent_callback(query: str) -> str:
    """
    Callback function for the Red Team Agent to interact with the target agent.
    
    Args:
        query: The adversarial prompt to send to the agent
        
    Returns:
        The agent's response to the query
    """
    # Create a message in the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id, 
        role="user", 
        content=query
    )
    
    # Create a run to process the message
    run = project_client.agents.runs.create(
        thread_id=thread.id, 
        agent_id=agent.id
    )

    # Poll the run until it completes
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
        print(f"Run status: {run.status}")

    # Handle errors
    if run.status == "failed":
        print(f"Run error: {run.last_error}")
        return "Error: Agent run failed."
    
    # Retrieve and return the agent's response
    messages = project_client.agents.messages.list(
        thread_id=thread.id, 
        order=ListSortOrder.DESCENDING
    )
    
    for msg in messages:
        if msg.text_messages:
            return msg.text_messages[0].text.value
    
    return "Could not get a response from the agent."

print("✅ Agent callback function defined successfully!")

## Step 8: Configure the Red Team Agent

Initialize the AI Red Teaming Agent with your configuration:

- **azure_ai_project**: Your Azure AI Foundry project endpoint
- **credential**: Azure credentials for authentication
- **risk_categories**: Which risk types to test (Violence, Sexual, Hate/Unfairness, Self-Harm)
- **num_objectives**: Number of attack objectives per risk category
- **output_dir**: Where to save scan results

### Supported Risk Categories

| Risk Category | Description |
|--------------|-------------|
| **Violence** | Content intended to hurt, injure, damage, or kill |
| **Sexual** | Content related to anatomical organs, romantic relationships, or sexual acts |
| **Hate and Unfairness** | Hateful content or unfair representations of social groups |
| **Self-Harm** | Content related to actions intended to hurt or kill oneself |

> **Learn More**: [Supported Risk Categories](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/ai-red-teaming-agent#supported-risk-categories)

For this initial scan, we'll test **Violence** risk category with **1 attack objective** to keep it quick.

In [ ]:
# Initialize the Red Team Agent
red_team = RedTeam(
    azure_ai_project=project_endpoint,
    credential=credential,
    risk_categories=[RiskCategory.Violence],  # Testing for violent content
    num_objectives=1,  # One attack objective per category
    output_dir="scan-results/"  # Save results to this directory
)

print("✅ Red Team Agent configured successfully!")
print(f"📊 Risk Categories: Violence")
print(f"🎯 Attack Objectives per Category: 1")
print(f"💾 Output Directory: scan-results/")

## Step 9: Run the Red Team Scan

Now we'll execute the red team scan with a specific **attack strategy**. The scan will:

1. Generate adversarial prompts based on the risk category
2. Apply the attack strategy to transform prompts
3. Send transformed prompts to your agent via the callback
4. Evaluate responses for successful attacks
5. Calculate metrics like Attack Success Rate (ASR)

### Supported Attack Strategies

Some common attack strategies include:

| Strategy | Description |
|----------|-------------|
| **Flip** | Flips characters from front to back, creating a mirrored effect |
| **Base64** | Encodes prompts in Base64 format |
| **Leetspeak** | Replaces letters with similar-looking numbers or symbols |
| **Jailbreak** | Injects prompts to bypass AI safeguards (UPIA) |
| **ROT13** | Shifts characters by 13 positions |

> **Learn More**: [Supported Attack Strategies](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/ai-red-teaming-agent#supported-attack-strategies)

For this scan, we'll use the **Flip** strategy.

> ⏱️ **Note**: This may take a few minutes depending on the number of objectives and attack strategies.

In [ ]:
print("🚀 Starting Red Team scan...")
print(f"Target: {agent.name}")
print(f"Attack Strategy: Flip")
print("-" * 50)

# Run the scan
result = await red_team.scan(
    target=agent_callback,
    scan_name="1-Agent-Target",
    attack_strategies=[AttackStrategy.Flip],
)

print("-" * 50)
print("✅ Red Team scan complete!")
print(f"📁 Results saved to: scan-results/")

## Step 10: Clean Up Resources

Close the project client to release resources properly.

In [ ]:
# Close the project client
project_client.close()

print("✅ Resources cleaned up successfully!")

## Understanding Your Results

After the scan completes, you'll find a JSON scorecard in the `scan_results/` directory containing:

### Key Metrics

- **Attack Success Rate (ASR)**: Percentage of attacks that successfully elicited unsafe responses
- **Attack Breakdown**: Results organized by risk category and attack complexity
- **Row-level Data**: Each attack-response pair with success indicators

### Viewing Results in Azure AI Foundry Portal

The scan results are also automatically logged to your Azure AI Foundry project. To view them:

1. Navigate to your [Azure AI Foundry project](https://ai.azure.com)
2. Go to the **Evaluation** page
3. Select **AI red teaming** tab
4. Click on your scan to see detailed reports

![AI Red Teaming Portal View](https://learn.microsoft.com/en-us/azure/ai-foundry/media/evaluations/red-teaming-agent/ai-red-team.png)

You can drill down into:
- **Risk Category Reports**: Breakdown of successful attacks per category
- **Attack Complexity Reports**: Classification by attack difficulty
- **Data View**: Row-level attack-response pairs with metadata
- **Conversation History**: Full context for each interaction

![AI Red Teaming Data View](https://learn.microsoft.com/en-us/azure/ai-foundry/media/evaluations/red-teaming-agent/ai-red-team-data.png)

> **Learn More**: [Viewing AI Red Teaming Results](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/run-scans-ai-red-teaming-agent#viewing-ai-red-teaming-results-in-azure-ai-foundry-project-preview)

---

## Next Steps

Congratulations! You've successfully run your first AI Red Teaming scan. 🎉

### What's Next?

- **Lab 2**: Learn to scan model endpoints directly
- **Lab 3**: Run red team scans in the cloud
- **Lab 4**: Explore advanced scanning techniques

### Additional Resources

- 📚 [AI Red Teaming Agent Concepts](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/ai-red-teaming-agent)
- 🛠️ [PyRIT - Python Risk Identification Tool](https://github.com/Azure/PyRIT)
- 🔒 [Planning Red Teaming for LLMs](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/concepts/red-teaming)
- 📊 [Risk and Safety Evaluations](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-metrics-built-in#risk-and-safety-evaluators)

### Try These Variations

1. **Test more risk categories**: Add `RiskCategory.Sexual`, `RiskCategory.HateUnfairness`, `RiskCategory.SelfHarm`
2. **Increase attack objectives**: Set `num_objectives=10` for more comprehensive testing
3. **Try different attack strategies**: Use `AttackStrategy.Jailbreak`, `AttackStrategy.Base64`, etc.
4. **Custom attack prompts**: Bring your own attack objectives specific to your domain

Keep building safer AI! 🚀